In [42]:
import geopandas as gpd
import pandas as pd
import segregation
import libpysal
import numpy as np
# from segregation.compute_all import ComputeAllAspatialSegregation, ComputeAllSpatialSegregation, ComputeAllSegregation
from segregation.batch import batch_compute_multigroup
from nycschools import snapshot
from nycschools.segregation import entropy
from nycschools.segregation import mutual_information as M


In [60]:
from math import log2
groups = 8
h = -log2(1/groups)
h
log2(8)

3.0

In [56]:

groups = ["black", "white", "asian", "hispanic"]
schools = ["a", "b", "c", "d"]

# Create a list to hold the data
data = []

# Loop through each school and group, adding 25 people for each combination
for school in schools:
    for group in groups:
        data.append({"school": school, "group": group, "people": np.random.randint(25, 100)})

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)
# display(df)
M(df,group="group",unit="school",n="people")
df = df.pivot(index="school", columns="group", values="people").reset_index()
df.name=None
df
even = [50,50,50,50]
asian, black, hispanic, white = even
N = sum(even)
H = -sum([p/N*np.log2(p/N) for p in even])
print(f"Entropy even distribution: {H}")

uneven = [40,60,30,70]
asian, black, hispanic, white = uneven
N = sum(uneven)
H = -sum([p/N*np.log2(p/N) for p in uneven])
print(f"Entropy uneven distribution: {H}")

Entropy even distribution: 2.0
Entropy uneven distribution: 1.9261207468426804


In [15]:
df = snapshot.load_snapshots()
df = df[df.ay == df.ay.max()]
groups = [ 'asian_n', 'black_n', 'hispanic_n', 'amerindian_n', 'white_n', ]
city = batch_compute_multigroup(df, groups=groups,)
city.reset_index(inplace=True)
city["District"] = "nyc"
city = city.pivot(index="District", columns='Name', values='Statistic').reset_index()
city.columns.name = None
city

,District,GlobalDistortion,MultiDissim,MultiDivergence,MultiDiversity,MultiGini,MultiInfoTheory,MultiNormExposure,MultiRelativeDiversity,MultiSquaredCoefVar,SimpsonsConcentration,SimpsonsInteraction
0,nyc,82.265,0.5458,0.4525,1.3461,0.7002,0.3361,0.3084,0.3111,0.2536,0.2915,0.7085


In [18]:

districts = df[groups  + ["district"]].groupby('district').sum()
districts = districts.reset_index()


def district_multi(row):
    district = batch_compute_multigroup(row, groups=groups)
    district.reset_index(inplace=True)
    district = district.pivot(index="District", columns='Name', values='Statistic').reset_index()
    district.columns.name = None
    # concat district to the row
    row = pd.concat([district, row[["district"]]], axis=1)
    return row
districts.apply(district_multi, axis=1)

TypeError: `data` must be a geopanads.GeoDataFrame with a vaild geometry column